In [1]:
library(plm)
library(lmtest)
library(sandwich)
library(ggplot2)
library(zoo)

Lade n"otiges Paket: zoo


Attache Paket: 'zoo'


Die folgenden Objekte sind maskiert von 'package:base':

    as.Date, as.Date.numeric




In [2]:
data <- read.csv("DF_FINAL.csv")

In [3]:
# Indexing based on country and year
pdata <- pdata.frame(data, index = c("country" , "year"))

In [4]:
head(pdata)

,Index,country,year,Lnrgdpo,Lnlabor_stock,Lncapital_stock,RL,GS,RE,MO,LnRL,LnGS,LnRE,LnMO
,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Austria-2000,0,Austria,2000,12.63437,9.955952,14.40257,82.5,34.60,79.20000,74.33333,4.412798,3.543854,4.371976,4.308559
Austria-2001,1,Austria,2001,12.61141,9.960741,14.42777,83.0,34.60,79.80000,72.66667,4.418841,3.543854,4.379524,4.285883
Austria-2002,2,Austria,2002,12.63228,9.963124,14.44894,83.5,31.70,78.20000,73.20000,4.424847,3.456317,4.359270,4.293195
Austria-2003,3,Austria,2003,12.65686,9.970033,14.47125,84.0,31.95,77.55000,73.80000,4.430817,3.464172,4.350923,4.301359
Austria-2004,4,Austria,2004,12.69611,9.981648,14.49256,84.0,32.25,77.95000,73.26667,4.430817,3.473518,4.356068,4.294106
Austria-2005,5,Austria,2005,12.78654,9.982874,14.51276,85.0,33.50,76.86667,73.40000,4.442651,3.511545,4.342072,4.295924


---
# **Formal tests**

In [5]:
# Estimating the pooled and random-effects models
Pooled <- plm(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "pooling")
RE <- plm(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "random")

### Breusch-Pagan test for homoskedasticity

In [6]:
# Breusch-Pagan test for homoskedasticity
bptest(Pooled)


	studentized Breusch-Pagan test

data:  Pooled
BP = 41.8, df = 6, p-value = 2.013e-07


### Breusch-Godfrey test for serial correlation

In [7]:
# Breusch-Godfrey/Wooldridge test for serial correlation
pbgtest(Pooled)


	Breusch-Godfrey/Wooldridge test for serial correlation in panel models

data:  Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO
chisq = 479.32, df = 20, p-value < 2.2e-16
alternative hypothesis: serial correlation in idiosyncratic errors


### Pesaran's CD test for cross-sectional dependence

In [8]:
# Pesaran's CD test for cross-sectional dependence
pcdtest(Pooled , test = "cd")


	Pesaran CD test for cross-sectional dependence in panels

data:  Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO
z = 34.211, p-value < 2.2e-16
alternative hypothesis: cross-sectional dependence


### Hausman test for RE/FE

In [ ]:
# Robust Hausman test
form <- Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO
phtest(form , data = pdata , method = "aux" , vcov = vcovSCC)


	Regression-based Hausman test, vcov: vcovSCC

data:  form
chisq = 127.2, df = 6, p-value < 2.2e-16
alternative hypothesis: one model is inconsistent


### Cross-sectionally augmented Im, Pesaran, and Shin (IPS) test for unit roots in panel models

In [ ]:
# Loading the differenced dataset
df <- read.csv("DF_FINAL_diff.csv")
df <- pdata.frame(df, index = c("country" , "year"))
head(df)

,Index,country,year,Lnrgdpo,Lnlabor_stock,Lncapital_stock,RL,GS,RE,MO,...,LnGS,LnRE,LnMO,d_Lnrgdpo,d_Lnlabor_stock,d_Lncapital_stock,d_RL,d_GS,d_RE,d_MO
,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Austria-2001,1,Austria,2001,12.61141,9.960741,14.42777,83.0,34.60,79.80000,72.66667,...,3.543854,4.379524,4.285883,-0.02295222,0.004789680,0.02519178,0.5,0.00,0.600000,-1.6666667
Austria-2002,2,Austria,2002,12.63228,9.963124,14.44894,83.5,31.70,78.20000,73.20000,...,3.456317,4.359270,4.293195,0.02086782,0.002382666,0.02116974,0.5,-2.90,-1.600000,0.5333333
Austria-2003,3,Austria,2003,12.65686,9.970033,14.47125,84.0,31.95,77.55000,73.80000,...,3.464172,4.350923,4.301359,0.02457348,0.006909498,0.02231303,0.5,0.25,-0.650000,0.6000000
Austria-2004,4,Austria,2004,12.69611,9.981648,14.49256,84.0,32.25,77.95000,73.26667,...,3.473518,4.356068,4.294106,0.03925263,0.011614352,0.02131161,0.0,0.30,0.400000,-0.5333333
Austria-2005,5,Austria,2005,12.78654,9.982874,14.51276,85.0,33.50,76.86667,73.40000,...,3.511545,4.342072,4.295924,0.09042667,0.001226535,0.02019777,1.0,1.25,-1.083333,0.1333333
Austria-2006,6,Austria,2006,12.80598,9.994661,14.53242,87.0,36.70,80.36667,74.13333,...,3.602777,4.386599,4.305865,0.01944595,0.011787026,0.01965728,2.0,3.20,3.500000,0.7333333


In [ ]:
# Original variables
CIPS_Lnrgdpo <- cipstest(df$Lnrgdpo , lags=1)
print(CIPS_Lnrgdpo)
CIPS_Lnlabor_stock <- cipstest(df$Lnlabor_stock , lags=1)
print(CIPS_Lnlabor_stock)
CIPS_Lncapital_stock <- cipstest(df$Lncapital_stock , lags=1)
print(CIPS_Lncapital_stock)
CIPS_RL <- cipstest(df$RL , lags=1)
print(CIPS_RL)
CIPS_GS <- cipstest(df$GS , lags=1)
print(CIPS_GS)
CIPS_RE <- cipstest(df$RE , lags=1)
print(CIPS_RE)
CIPS_MO <- cipstest(df$MO , lags=1)
print(CIPS_MO)


	Pesaran's CIPS test for unit roots

data:  df$Lnrgdpo
CIPS test = -2.8644, lag order = 1, p-value = 0.01231
alternative hypothesis: Stationarity


	Pesaran's CIPS test for unit roots

data:  df$Lnlabor_stock
CIPS test = -2.6343, lag order = 1, p-value = 0.08016
alternative hypothesis: Stationarity



Warning message in cipstest(df$Lncapital_stock, lags = 1):
"p-value greater than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$Lncapital_stock
CIPS test = -2.4035, lag order = 1, p-value = 0.1
alternative hypothesis: Stationarity



Warning message in cipstest(df$RL, lags = 1):
"p-value greater than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$RL
CIPS test = -1.2065, lag order = 1, p-value = 0.1
alternative hypothesis: Stationarity



...


	Pesaran's CIPS test for unit roots

data:  df$GS
CIPS test = -2.4953, lag order = 1, p-value = 0.1
alternative hypothesis: Stationarity



Warning message in cipstest(df$RE, lags = 1):
"p-value greater than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$RE
CIPS test = -2.2975, lag order = 1, p-value = 0.1
alternative hypothesis: Stationarity



Warning message in cipstest(df$MO, lags = 1):
"p-value greater than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$MO
CIPS test = -2.5472, lag order = 1, p-value = 0.1
alternative hypothesis: Stationarity



In [ ]:
CIPS_Lnrgdpo <- cipstest(df$d_Lnrgdpo , lags=1)
print(CIPS_Lnrgdpo)
CIPS_Lnlabor_stock <- cipstest(df$d_Lnlabor_stock , lags=1)
print(CIPS_Lnlabor_stock)
CIPS_Lncapital_stock <- cipstest(df$d_Lncapital_stock , lags=1)
print(CIPS_Lncapital_stock)
CIPS_RL <- cipstest(df$d_RL , lags=1)
print(CIPS_RL)
CIPS_GS <- cipstest(df$d_GS , lags=1)
print(CIPS_GS)
CIPS_RE <- cipstest(df$d_RE , lags=1)
print(CIPS_RE)
CIPS_MO <- cipstest(df$d_MO , lags=1)
print(CIPS_MO)

Warning message in cipstest(df$d_Lnrgdpo, lags = 1):
"p-value smaller than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$d_Lnrgdpo
CIPS test = -3.2089, lag order = 1, p-value = 0.01
alternative hypothesis: Stationarity


	Pesaran's CIPS test for unit roots

data:  df$d_Lnlabor_stock
CIPS test = -2.7646, lag order = 1, p-value = 0.03421
alternative hypothesis: Stationarity


	Pesaran's CIPS test for unit roots

data:  df$d_Lncapital_stock
CIPS test = -2.665, lag order = 1, p-value = 0.06428
alternative hypothesis: Stationarity


	Pesaran's CIPS test for unit roots

data:  df$d_RL
CIPS test = -2.8567, lag order = 1, p-value = 0.01416
alternative hypothesis: Stationarity



Warning message in cipstest(df$d_GS, lags = 1):
"p-value smaller than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$d_GS
CIPS test = -3.3983, lag order = 1, p-value = 0.01
alternative hypothesis: Stationarity



Warning message in cipstest(df$d_RE, lags = 1):
"p-value smaller than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$d_RE
CIPS test = -3.1064, lag order = 1, p-value = 0.01
alternative hypothesis: Stationarity



Warning message in cipstest(df$d_MO, lags = 1):
"p-value smaller than printed p-value"



	Pesaran's CIPS test for unit roots

data:  df$d_MO
CIPS test = -3.2233, lag order = 1, p-value = 0.01
alternative hypothesis: Stationarity



---
## Testing for entity and time effects of the FE model

In [ ]:
# FE with entity and FE with time
FE_entity <- plm(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "within", effect = "individual")
FE_time <- plm(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "within", effect = "time")

In [ ]:
# Test for significance of entity effects
pFtest(FE_entity , Pooled)


	F test for individual effects

data:  Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO
F = 96.805, df1 = 29, df2 = 564, p-value < 2.2e-16
alternative hypothesis: significant effects


In [ ]:
# Test for significance of time effects
pFtest(FE_time , Pooled)


	F test for time effects

data:  Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO
F = 5.5282, df1 = 19, df2 = 574, p-value = 1.304e-12
alternative hypothesis: significant effects


---
# **Regular estimations**

## Fixed-effects estimation with both entity and time effects
$$
y_{i,t} = \mu_{i} + \lambda_{t} + \beta x_{i,t} + \epsilon_{i,t}

In [ ]:
# Fixed-Effects Estimation with both entity and time effects for comparison
FE <- plm(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "within", effect = "twoways")

In [ ]:
summary(FE, vcov = vcovSCC)

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
plm(formula = Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + 
    GS + RE + MO, data = pdata, effect = "twoways", model = "within")

Balanced Panel: n = 30, T = 20, N = 600

Residuals:
       Min.     1st Qu.      Median     3rd Qu.        Max. 
-2.2628e-01 -3.4544e-02 -6.4836e-05  3.4859e-02  2.7091e-01 

Coefficients:
                   Estimate  Std. Error t-value  Pr(>|t|)    
Lnlabor_stock    0.33551040  0.16304679  2.0578  0.040089 *  
Lncapital_stock  0.51212106  0.06238541  8.2090 1.622e-15 ***
RL               0.00445198  0.00102049  4.3626 1.537e-05 ***
GS               0.00269215  0.00067069  4.0140 6.806e-05 ***
RE               0.00150726  0.00093555  1.6111  0.107737    
MO              -0.00233544  0.00071387 -3.2715  0.001137 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Total Sum of Squares:    4.6764
Residual Sum of Squares: 2.7592
R-Squared:

---
## Pooled common correlated effects estimator (CCEP)
$$
y_{i,t} = \mu_{i} + \beta x_{i,t} + \gamma f_{t} + \epsilon_{i,t}
$$

In [ ]:
# Pooled Common Correlated Effects Estimator (PCCE)
CCE_P <- pcce(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "p")

In [ ]:
summary(CCE_P, vcov = vcovSCC)

Common Correlated Effects Pooled model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + 
    GS + RE + MO, data = pdata, model = "p")

Balanced Panel: n = 30, T = 20, N = 600

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.1456495706 -0.0122372469 -0.0009223022  0.0122074445  0.1543753008 

Coefficients:
                   Estimate  Std. Error z-value  Pr(>|z|)    
Lnlabor_stock    0.57929764  0.06266540  9.2443 < 2.2e-16 ***
Lncapital_stock  0.77433759  0.08012978  9.6635 < 2.2e-16 ***
RL              -0.00078756  0.00030381 -2.5923  0.009534 ** 
GS               0.00063116  0.00035506  1.7776  0.075467 .  
RE               0.00113013  0.00042973  2.6299  0.008542 ** 
MO              -0.00013673  0.00040453 -0.3380  0.735357    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Total Sum of Squares: 1319.8
Residual Sum of Squares: 0.43209
HPY R-s

---
## Mean groups common correlated effects estimator (CCEMG)
$$
y_{i,t} = \mu_{i} + \beta_{i} x_{i,t} + \gamma f_{t} + \epsilon_{i,t}

In [ ]:
# Mean Groups Common Correlated Effects Estimator (MGCCE)
CCE_MG <- pcce(Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + GS + RE + MO, data = pdata, model = "mg")

In [ ]:
summary(CCE_MG , vcov = vcovSCC)

Common Correlated Effects Mean Groups model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = Lnrgdpo ~ Lnlabor_stock + Lncapital_stock + RL + 
    GS + RE + MO, data = pdata, model = "mg")

Balanced Panel: n = 30, T = 20, N = 600

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.0489473497 -0.0065300722  0.0001165177  0.0063679202  0.0560125446 

Coefficients:
                   Estimate  Std. Error  z-value  Pr(>|z|)    
Lnlabor_stock    3.5690e-01  2.9839e-02  11.9607 < 2.2e-16 ***
Lncapital_stock  2.3493e-01  4.9076e-02   4.7870 1.693e-06 ***
RL              -1.7567e-03  9.7457e-05 -18.0253 < 2.2e-16 ***
GS               4.9765e-04  1.8389e-04   2.7062  0.006805 ** 
RE              -1.3785e-03  2.5933e-04  -5.3156 1.063e-07 ***
MO               5.2399e-04  2.0782e-04   2.5213  0.011691 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Total Sum of Squares: 1319.8
Residual Sum of Squares: 0.

---


# **First-differenced estimations**

In [ ]:
data <- read.csv("DF_DIFF.csv")
# Indexing based on country and year
pdata <- pdata.frame(data, index = c("country" , "year"))
head(pdata)

,Index,country,year,delta_Lnrgdpo,delta_Lnlabor_stock,delta_Lncapital_stock,delta_RL,delta_GS,delta_RE,delta_MO
,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Austria-2001,1,Austria,2001,-0.02295222,0.004789680,0.02519178,0.5,0.00,0.600000,-1.6666667
Austria-2002,2,Austria,2002,0.02086782,0.002382666,0.02116974,0.5,-2.90,-1.600000,0.5333333
Austria-2003,3,Austria,2003,0.02457348,0.006909498,0.02231303,0.5,0.25,-0.650000,0.6000000
Austria-2004,4,Austria,2004,0.03925263,0.011614352,0.02131161,0.0,0.30,0.400000,-0.5333333
Austria-2005,5,Austria,2005,0.09042667,0.001226535,0.02019777,1.0,1.25,-1.083333,0.1333333
Austria-2006,6,Austria,2006,0.01944595,0.011787026,0.01965728,2.0,3.20,3.500000,0.7333333


---
## Fixed-effects estimation with both entity and time effects
$$   \Delta {ln(GDP)}_{i,t} = \mu_{i} + \lambda_{t} + \beta_{1} \Delta {ln(L)}_{i,t} + \beta_{2} \Delta {ln(K)}_{i,t} + \beta_{3} \Delta {RL}_{i,t} + \beta_{4} \Delta {GS}_{i,t} + \beta_{5} \Delta {RE}_{i,t} + \beta_{6} \Delta {MO}_{i,t} + \epsilon_{i,t}$$

In [ ]:
# Fixed-Effects Estimation with both entity and time effects
FE_diff <- plm(delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_Lncapital_stock + delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, model = "within", effect = "twoways")
summary(FE_diff, vcov = vcovSCC)

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
plm(formula = delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_Lncapital_stock + 
    delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, 
    effect = "twoways", model = "within")

Balanced Panel: n = 30, T = 19, N = 570

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.1929489 -0.0159077 -0.0002493  0.0159833  0.1897883 

Coefficients:
                         Estimate  Std. Error t-value  Pr(>|t|)    
delta_Lnlabor_stock    0.53297869  0.09126483  5.8399 9.249e-09 ***
delta_Lncapital_stock  0.54551106  0.12082168  4.5150 7.852e-06 ***
delta_RL              -0.00061515  0.00064102 -0.9596 0.3376835    
delta_GS               0.00084452  0.00021709  3.8902 0.0001133 ***
delta_RE              -0.00034349  0.00072746 -0.4722 0.6369994    
delta_MO              -0.00071769  0.00048845 -1.4693 0.1423596    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' 

---
## Pooled common correlated effects estimator (CCEP)
$$
\Delta ln(GDP)_{i,t} = \mu + \lambda_{t} + \beta_{1} \Delta ln(L)_{i,t} + \beta_{2} \Delta ln(K)_{i,t} + \beta_{3} \Delta RL_{i,t} + \beta_{4} \Delta GS_{i,t} + \beta_{5} \Delta RE_{i,t} + \beta_{6} \Delta MO_{i,t} + \gamma f_{t} + \epsilon_{i,t}
$$
where
$$
\gamma f_{t} = \gamma_{1} \overline{\Delta ln(GDP)_{t}} + \gamma_{2} \overline{\Delta ln(L)_{t}} + \gamma_{3} \overline{\Delta ln(K)_{t}} + \gamma_{4} \overline{\Delta RL_{t}} + \gamma_{5} \overline{\Delta GS_{t}} + \gamma_{6} \overline{\Delta RE_{t}} + \gamma_{7} \overline{\Delta MO_{t}}

In [ ]:
# Pooled Common Correlated Effects Estimator (PCCE)
CCE_P_diff <- pcce(delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_Lncapital_stock + delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, model = "p")
summary(CCE_P_diff, vcov = vcovSCC)

Common Correlated Effects Pooled model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_Lncapital_stock + 
    delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, 
    model = "p")

Balanced Panel: n = 30, T = 19, N = 570

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.1376046867 -0.0121280307  0.0004819965  0.0116215832  0.1680225472 

Coefficients:
                         Estimate  Std. Error z-value  Pr(>|z|)    
delta_Lnlabor_stock    5.1336e-01  5.1402e-02  9.9873 < 2.2e-16 ***
delta_Lncapital_stock  7.5297e-01  2.7522e-01  2.7359  0.006221 ** 
delta_RL              -5.5603e-04  3.3163e-04 -1.6767  0.093606 .  
delta_GS               4.9221e-04  3.7196e-04  1.3233  0.185742    
delta_RE              -6.6243e-04  5.9921e-04 -1.1055  0.268941    
delta_MO              -7.4681e-05  5.3062e-04 -0.1407  0.888073    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*'

---
## Mean groups common correlated effects estimator (CCEMG)
For each cross-section $i$ :
$$
\Delta ln(GDP)_{i,t} = \mu_{i} + \lambda_{t} + \beta_{1,i} \Delta ln(L)_{i,t} + \beta_{2,i} \Delta ln(K)_{i,t} + \beta_{3,i} \Delta RL_{i,t} + \beta_{4,i} \Delta GS_{i,t} + \beta_{5,i} \Delta RE_{i,t} + \beta_{6,i} \Delta MO_{i,t} + \gamma_{i} f_{t} + \epsilon_{i,t}
$$
where
$$
 \gamma_i f_t = \gamma_{1,i} \overline{\Delta ln({GDP})_t} + \gamma_{2,i} \overline{\Delta ln({L})_t} + \gamma_{3,i} \overline{\Delta ln({K})_t} + \gamma_{4,i} \overline{\Delta {RL}_t} + \gamma_{5,i} \overline{\Delta {GS}_t} + \gamma_{6,i} \overline{\Delta {RE}_t} + \gamma_{7,i} \overline{\Delta {MO}_t} 
 $$

then, average the coefficients across all cross-sections:
$$
\beta_{j} = \frac{1}{N} \sum_{i=1}^{N} \beta{j,i} \quad \text{for } j=1 , 2 , \ldots , 6
$$

In [ ]:
# Mean Groups Common Correlated Effects Estimator (MGCCE)
CCE_MG_diff <- pcce(delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_Lncapital_stock + delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, model = "mg")
summary(CCE_MG_diff , vcov = vcovSCC)

Common Correlated Effects Mean Groups model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_Lncapital_stock + 
    delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, 
    model = "mg")

Balanced Panel: n = 30, T = 19, N = 570

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.0668382648 -0.0075895714  0.0003037069  0.0074156619  0.0728148834 

Coefficients:
                         Estimate  Std. Error z-value  Pr(>|z|)    
delta_Lnlabor_stock    0.16797182  0.03567114  4.7089 2.491e-06 ***
delta_Lncapital_stock  0.34195022  0.14286339  2.3935 0.0166863 *  
delta_RL              -0.00217232  0.00022638 -9.5958 < 2.2e-16 ***
delta_GS               0.00062688  0.00017441  3.5942 0.0003253 ***
delta_RE              -0.00230020  0.00030615 -7.5134 5.760e-14 ***
delta_MO               0.00136401  0.00030951  4.4070 1.048e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.

Sensitivity analysis of Lncapital_stock by running the regression again and excluding Lncapital_stock.

In [ ]:
# Mean Groups Common Correlated Effects Estimator (MGCCE) excluding Lncapital_stock
CCE_MG_diff2 <- pcce(delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, model = "mg")
summary(CCE_MG_diff2 , vcov = vcovSCC)

Common Correlated Effects Mean Groups model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = delta_Lnrgdpo ~ delta_Lnlabor_stock + delta_RL + 
    delta_GS + delta_RE + delta_MO, data = pdata, model = "mg")

Balanced Panel: n = 30, T = 19, N = 570

Residuals:
        Min.      1st Qu.       Median      3rd Qu.         Max. 
-0.071299775 -0.008747891  0.000175403  0.009186237  0.116119682 

Coefficients:
                       Estimate  Std. Error  z-value  Pr(>|z|)    
delta_Lnlabor_stock  0.40164806  0.04320314   9.2967 < 2.2e-16 ***
delta_RL            -0.00207695  0.00016544 -12.5538 < 2.2e-16 ***
delta_GS             0.00078698  0.00022573   3.4863 0.0004897 ***
delta_RE            -0.00186128  0.00034474  -5.3990 6.701e-08 ***
delta_MO             0.00168186  0.00035987   4.6735 2.960e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Total Sum of Squares: 1.374
Residual Sum of Squares: 0.21
HPY R-squared: 0.57879

Sensitivity analysis of Lncapital_stock by running the regression again and excluding Lnlabor_stock.

In [ ]:
# Mean Groups Common Correlated Effects Estimator (MGCCE) excluding Lnlabor_stock
CCE_MG_diff3 <- pcce(delta_Lnrgdpo ~ delta_Lncapital_stock + delta_RL + delta_GS + delta_RE + delta_MO, data = pdata, model = "mg")
summary(CCE_MG_diff3 , vcov = vcovSCC)

Common Correlated Effects Mean Groups model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = delta_Lnrgdpo ~ delta_Lncapital_stock + delta_RL + 
    delta_GS + delta_RE + delta_MO, data = pdata, model = "mg")

Balanced Panel: n = 30, T = 19, N = 570

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.1297959849 -0.0096360695  0.0008839333  0.0102689250  0.1085740904 

Coefficients:
                         Estimate  Std. Error z-value  Pr(>|z|)    
delta_Lncapital_stock -0.36624109  0.15678682 -2.3359    0.0195 *  
delta_RL              -0.00222444  0.00029896 -7.4406 1.002e-13 ***
delta_GS               0.00082037  0.00017394  4.7165 2.399e-06 ***
delta_RE              -0.00212929  0.00036601 -5.8176 5.971e-09 ***
delta_MO               0.00082963  0.00028980  2.8627    0.0042 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Total Sum of Squares: 1.374
Residual Sum of Squares: 0.24378
HPY R-sq

---
# Exporting estimations to CSV

In [ ]:
# Create a data frame to store the results
results <- data.frame(Model = character(),
                      FittedValues = numeric(),
                      Residuals = numeric(),
                      stringsAsFactors = FALSE)

In [ ]:
# Function to extract fitted values and residuals
extract_fitted_residuals <- function(model, model_name) {
  fitted_vals <- fitted(model)
  residuals_vals <- residuals(model)
  data.frame(Model = rep(model_name, length(fitted_vals)),
             FittedValues = fitted_vals,
             Residuals = residuals_vals)
}

In [ ]:
# Extract for each model and combine
results <- rbind(results, extract_fitted_residuals(Pooled, "Pooled"))
results <- rbind(results, extract_fitted_residuals(RE, "RE"))
results <- rbind(results, extract_fitted_residuals(FE, "FE"))
results <- rbind(results, extract_fitted_residuals(CCE_P, "CCE_P"))
results <- rbind(results, extract_fitted_residuals(CCE_MG, "CCE_MG"))
results <- rbind(results, extract_fitted_residuals(FE_diff, "FE_diff"))
results <- rbind(results, extract_fitted_residuals(CCE_P_diff, "CCE_P_diff"))
results <- rbind(results, extract_fitted_residuals(CCE_MG_diff, "CCE_MG_diff"))
results <- rbind(results, extract_fitted_residuals(CCE_MG_diff2, "CCE_MG_diff2"))
results <- rbind(results, extract_fitted_residuals(CCE_MG_diff3, "CCE_MG_diff3"))

In [ ]:
head(results)

,Model,FittedValues,Residuals
,<chr>,<dbl>,<dbl>
Austria-2000,Pooled,12.80495,-0.1705837
Austria-2001,Pooled,12.82489,-0.2134796
Austria-2002,Pooled,12.84021,-0.2079320
Austria-2003,Pooled,12.86217,-0.2053102
Austria-2004,Pooled,12.87837,-0.1822627
Austria-2005,Pooled,12.90625,-0.1197160


In [ ]:
# Export to CSV
write.csv(results, file = "R-models.csv", row.names = FALSE)

In [ ]:
MG_country_coefficients <- CCE_MG$indcoef
MG_diff_country_coefficients <- CCE_MG_diff$indcoef

In [ ]:
write.csv(MG_country_coefficients, file = "MG_country_coefficients.csv")
write.csv(MG_diff_country_coefficients, file = "MG_diff_country_coefficients.csv")